In [4]:
# Install compatible versions of torch, torchvision, torchaudio, and transformers
!pip uninstall -y torch torchvision torchaudio transformers bitsandbytes
!pip uninstall transformers -y  # Remove the existing installation
!pip install -q ipywidgets
!pip install -q accelerate
!pip install -q jupyter
!pip install -q 'bitsandbytes-cuda113'  # Make sure to install the correct bitsandbytes version for CUDA 11.3

!pip install -q torch==2.3.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers==4.41.2
!pip install -q datasets  # Install the datasets library
!pip install -q pyarrow #==14.0.1  # Install compatible version of pyarrow
!pip install -q requests #==2.31.0  # Install compatible version of requests
!pip install peft

Found existing installation: torch 2.3.0+cu121
Uninstalling torch-2.3.0+cu121:
  Successfully uninstalled torch-2.3.0+cu121
Found existing installation: torchvision 0.18.0+cu121
Uninstalling torchvision-0.18.0+cu121:
  Successfully uninstalled torchvision-0.18.0+cu121
Found existing installation: torchaudio 2.3.0+cu121
Uninstalling torchaudio-2.3.0+cu121:
  Successfully uninstalled torchaudio-2.3.0+cu121
Found existing installation: transformers 4.41.2
Uninstalling transformers-4.41.2:
  Successfully uninstalled transformers-4.41.2
Found existing installation: bitsandbytes 0.43.1
Uninstalling bitsandbytes-0.43.1:
  Successfully uninstalled bitsandbytes-0.43.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 9.2 MB/s eta 0:00:00
ERROR: 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!nvcc --version
!pip install -q bitsandbytes

import json, urllib
!gdown 1JowBt1I-ljvkWdaBh0QE7oNMGbfkkAC5

import bitsandbytes as bnb
bnb.__version__  # This won't tell you the CUDA version, but ensures it's imported

# This line checks if CUDA is available in PyTorch, which indirectly helps ensure bitsandbytes is set up correctly.
import torch
print("CUDA available:", torch.cuda.is_available())
import transformers
#from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig#
help(transformers)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Downloading...
From: https://drive.google.com/uc?id=1JowBt1I-ljvkWdaBh0QE7oNMGbfkkAC5
To: /content/training12b_new.json
100% 39.9M/39.9M [00:00<00:00, 216MB/s]
CUDA available: True


In [ ]:
# Continue after restarting the runtime
import json
import urllib
import bitsandbytes as bnb
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

print("CUDA available:", torch.cuda.is_available())

# Check CUDA version and bitsandbytes installation
!nvcc --version
print("Bitsandbytes version:", bnb.__version__)

# Setup model function with GPU check
def setup_model(model_id):
    if not torch.cuda.is_available():
        raise RuntimeError("No GPU found. A GPU is needed for quantization.")

    device = "cuda"
    tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto"  # Use automatic device mapping
    )

    # Prepare the model for k-bit training and add LoRA adapters
    model = prepare_model_for_kbit_training(model)  # Prepare for k-bit training

    peft_config = LoraConfig(
        task_type="CAUSAL_LM",
        r=16,
        lora_alpha=32,
        lora_dropout=0.05,
        bias="none"
    )
    model = get_peft_model(model, peft_config)  # Add LoRA adapters

    # Ensure all necessary parameters are set to require gradients
    for param in model.parameters():
        if param.dtype in [torch.float16, torch.float32, torch.bfloat16, torch.complex64, torch.complex128]:
            param.requires_grad = True

    if tokenizer.pad_token:
        model.resize_token_embeddings(len(tokenizer))

    model.print_trainable_parameters()
    return model, tokenizer, device


def preprocess_for_training(examples, tokenizer):
    inputs = [f"Question: {q['body']}" for q in examples['questions']]
    ideal_answers = [q['ideal_answer'][0] for q in examples['questions']]

    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(ideal_answers, max_length=100, truncation=True, padding='max_length')

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

def predict_answers(model, tokenizer, questions, device):
    model.eval()
    predictions = []
    for question in questions:
        prompt = f"Question: {question['body']}\nAnswer:"
        inputs = tokenizer(prompt, return_tensors='pt').to(device)
        outputs = model.generate(inputs['input_ids'], max_new_tokens=50)
        answer = tokenizer.decode(outputs[0], skip_special_tokens=True).strip().split('Answer: ')[-1]
        predictions.append(answer)
    return predictions

if __name__ == "__main__":
    model_id = 'BioMistral/BioMistral-7B'
    data_path = '/content/training12b_new.json'

    # Load data and split into training and test sets
    with open(data_path, 'r') as f:
        data = json.load(f)
    train_questions, test_questions = train_test_split(data['questions'], test_size=0.2, random_state=42)
    train_data = {'questions': train_questions}
    test_data = {'questions': test_questions}

    model, tokenizer, device = setup_model(model_id)

    train_dataset = Dataset.from_dict(preprocess_for_training(train_data, tokenizer))
    test_dataset = Dataset.from_dict(preprocess_for_training(test_data, tokenizer))

    dataset_dict = DatasetDict({
        'train': train_dataset,
        'test': test_dataset
    })

    # Use DataCollatorForLanguageModeling to handle padding and alignment
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )

    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=1,
        per_device_train_batch_size=32,
        gradient_accumulation_steps=2,
        save_steps=200,
        save_total_limit=5,
        warmup_steps=300,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset_dict['train'],
        eval_dataset=dataset_dict['test'],
        tokenizer=tokenizer,
        data_collator=data_collator
    )

    trainer.train()

    # Predict answers for the test dataset
    predictions = predict_answers(model, tokenizer, test_data['questions'], device)

    print("Predictions:", predictions)


CUDA available: True
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Bitsandbytes version: 0.43.1


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


trainable params: 269,234,176 || all params: 7,248,556,032 || trainable%: 3.7143


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


Step,Training Loss
10,3.515400
20,3.286400
30,2.863300
40,2.558400
50,2.353800
60,2.147600


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attentio

In [1]:
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")

NameError: name 'model' is not defined

In [2]:
def compute_metrics(predictions, references):
    # Compute Success Accuracy (SAcc), Lenient Accuracy (LAcc), and Mean Reciprocal Rank (MRR)
    exact_matches = [1 if pred == ref else 0 for pred, ref in zip(predictions, references)]
    lenient_matches = [1 if ref in pred else 0 for pred, ref in zip(predictions, references)]

    ranks = []
    for pred, ref in zip(predictions, references):
        try:
            rank = pred.split().index(ref) + 1
            ranks.append(1 / rank)
        except ValueError:
            ranks.append(0)

    sacc = np.mean(exact_matches)
    lacc = np.mean(lenient_matches)
    mrr = np.mean(ranks)

    return sacc, lacc, mrr

# Extract references
references = [q['ideal_answer'][0] for q in test_data['questions']]

# Compute metrics
sacc, lacc, mrr = compute_metrics(predictions, references)

print(f"Success Accuracy (SAcc): {sacc:.4f}")
print(f"Lenient Accuracy (LAcc): {lacc:.4f}")
print(f"Mean Reciprocal Rank (MRR): {mrr:.4f}")

NameError: name 'test_data' is not defined